### [Oregon Curriculum Network](http://www.4dsolutions.net/ocn) <br />
[Discovering Math with Python](Introduction.ipynb)


# Chapter 11: QUATERNIONS

Quaternions were invented by Sir William Rowan Hamilton around 1843 and were considered a breakthrough.  

In subsequent chapters Willard Gibbs and Oliver Heaviside came up with a vector concept that proved easier to use for many of the physics applications for which quaternions had originally been proposed.

However quaternions and vectors together have become chief tools in accomplishing rotation.  Quaternions have some advantages over rotation matrices.  They're used a lot in physics engines, computer games, and robotics, whenever rotations are necessary.

You might think of quaternions as "vectors on steroids" if that helps.  They have some properties in common with vectors, and are conceived to have a "vector part" somewhat like complex numbers have a "real part".

Unit quaternions (w, x, y, z) with w\*\*2 + x\*\*2 + y\*\*2 + z\*\*2 == 1, form a group under multiplication.  Any two unit quaternions, when multiplied, produce a unit quaternion (Closure), and every unit quaternion has an inverse (w, -x, -y, -z), such that q \* q\*\*-1 gives the unit quaternion (1, 0, 0, 0).

In [ ]:
class Quaternion:
    """
    skeletal outline of the method's we'll need to 
    accomplish rotation
    """
    
    def __init__(self, degrees, v):
        """
        v is a Vector, the axis of rotation
        """
        pass
    
    def __mul__(self, other):
        """when other is a Vector, w == 0"""
        pass
    
    def __invert__(self, other):
        pass
    
    def __repr__(self):
        pass

What we'll plan to do, when rotating polyhedron P by n degrees around v = Vector(a, b, c), is initialize the corresponding unit quaternion (n, v) and then multiply every vector of P in a "sandwich" between that unit quaternion and its inverse:  

new_v = q \* old_v \* q\*\*-1.

Back to Chapter 10: [Complex Numbers](Complex%20Numbers.ipynb) <br />
Continue to Chapter 12: [The Mandelbrot Set](Mandelbrot%20Set.ipynb)<br />
[Introduction / Table of Contents](Introduction.ipynb)